In [42]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc
from sqlalchemy.sql import func, distinct, exists
from sqlalchemy import union, union_all

engine = create_engine('sqlite+pysqlite:////Users/vladislavvorosilov/Desktop/Python_SQL/Python_SQL_PI_2021_22_s2/Data/Students.db')
metadata_obj = MetaData()
metadata_obj.reflect(bind=engine) 
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")
conn=engine.connect()


## 1. Студентам 1 курса присвойте порядковые номера по дате рождения.


In [11]:
stmt = select(func.row_number().over(order_by=student.c.birthday), student).select_from(student)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT row_number() OVER (ORDER BY student.birthday) AS anon_1, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student
90


[(1, 83, 'Сорокина', 'Ирина', 200, 3, 'Усмань', datetime.datetime(1993, 9, 28, 0, 0), 48),
 (2, 85, 'Казакова', 'Мария', 150, 4, 'Нальчик', datetime.datetime(1994, 2, 18, 0, 0), 32),
 (3, 112, 'Захаров', 'Софон', 150, 3, 'Мурманск', datetime.datetime(1994, 3, 19, 0, 0), 46),
 (4, 94, 'Васильева', 'Оксана', 0, 1, 'Кизилюрт', datetime.datetime(1994, 5, 14, 0, 0), 32),
 (5, 113, 'Аксенов', 'Валерий', 150, 1, 'Мурманск', datetime.datetime(1994, 10, 27, 0, 0), 10),
 (6, 78, 'Агафонов', 'Василий', 150, 1, 'Грозный', datetime.datetime(1994, 10, 29, 0, 0), 10),
 (7, 95, 'Сорокина', 'Валерия', 150, 1, 'Енисейск', datetime.datetime(1995, 3, 1, 0, 0), 45),
 (8, 103, 'Трофимов', 'Владислав', 250, 5, 'Верхоянск', datetime.datetime(1995, 3, 24, 0, 0), 11),
 (9, 90, 'Костина', 'Кира', 0, 4, 'Устюжна', datetime.datetime(1995, 7, 15, 0, 0), 14),
 (10, 99, 'Борисова', 'Марина', 150, 1, 'Рославль', datetime.datetime(1995, 8, 7, 0, 0), 10)]

## 2. Для каждого университета ранжируйте курсы по числу студентов на курсе. Результат должен содержать название университета, номер курса, число студентов на курсе, ранг курса. Меньшее число ранга присвойте курсу с большим числом студентов.

In [40]:
subq = select(university.c.univ_id, university.c.univ_name, student.c.kurs, func.count(student.c.student_id).label("student_amount"))
subq = subq.select_from(university).join(student, student.c.univ_id == university.c.univ_id)
subq = subq.group_by(university.c.univ_id,student.c.kurs).subquery()

stmt = select(func.rank().over(partition_by = subq.c.univ_id, order_by = subq.c.student_amount), subq)
stmt = stmt.select_from(subq)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:15]

SELECT rank() OVER (PARTITION BY anon_2.univ_id ORDER BY anon_2.student_amount) AS anon_1, anon_2.univ_id, anon_2.univ_name, anon_2.kurs, anon_2.student_amount 
FROM (SELECT university.univ_id AS univ_id, university.univ_name AS univ_name, student.kurs AS kurs, count(student.student_id) AS student_amount 
FROM university JOIN student ON student.univ_id = university.univ_id GROUP BY university.univ_id, student.kurs) AS anon_2
44


[(1, 10, 'ВГУ', 4, 1),
 (2, 10, 'ВГУ', 2, 3),
 (2, 10, 'ВГУ', 5, 3),
 (4, 10, 'ВГУ', 1, 4),
 (5, 10, 'ВГУ', 3, 5),
 (1, 11, 'НГУ', 1, 1),
 (1, 11, 'НГУ', 3, 1),
 (1, 11, 'НГУ', 4, 1),
 (1, 11, 'НГУ', 5, 1),
 (1, 14, 'БГУ', 1, 1),
 (1, 14, 'БГУ', 2, 1),
 (3, 14, 'БГУ', 4, 2),
 (3, 14, 'БГУ', 5, 2),
 (5, 14, 'БГУ', 3, 3),
 (1, 15, 'ТГУ', 1, 1)]

In [41]:
stmt = select(func.dense_rank().over(partition_by = university.c.univ_id, order_by = func.count(student.c.student_id)), university.c.univ_name, student.c.kurs, func.count(student.c.student_id))
stmt = stmt.select_from(university).join(student, student.c.univ_id == university.c.univ_id)
stmt = stmt.group_by(university.c.univ_id,student.c.kurs)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:15]

SELECT dense_rank() OVER (PARTITION BY university.univ_id ORDER BY count(student.student_id)) AS anon_1, university.univ_name, student.kurs, count(student.student_id) AS count_1 
FROM university JOIN student ON student.univ_id = university.univ_id GROUP BY university.univ_id, student.kurs
44


[(1, 'ВГУ', 4, 1),
 (2, 'ВГУ', 2, 3),
 (2, 'ВГУ', 5, 3),
 (3, 'ВГУ', 1, 4),
 (4, 'ВГУ', 3, 5),
 (1, 'НГУ', 1, 1),
 (1, 'НГУ', 3, 1),
 (1, 'НГУ', 4, 1),
 (1, 'НГУ', 5, 1),
 (1, 'БГУ', 1, 1),
 (1, 'БГУ', 2, 1),
 (2, 'БГУ', 4, 2),
 (2, 'БГУ', 5, 2),
 (3, 'БГУ', 3, 3),
 (1, 'ТГУ', 1, 1)]

## 3. В университетах ВГУ и НГУ ранжируйте (без пропуска значений ранга) студентов по убыванию стипендии в рамках своего университета.

In [55]:
stmt = select(func.dense_rank().over(partition_by = university.c.univ_id, order_by = desc(student.c.stipend)), university.c.univ_name, student.c.kurs, student.c.surname, student.c.name, student.c.stipend)
stmt = stmt.select_from(university).join(student, student.c.univ_id == university.c.univ_id)
stmt = stmt.where(university.c.univ_name.in_(["ВГУ", "НГУ"]))
stmt = stmt.group_by(university.c.univ_id,student.c.student_id)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result

SELECT dense_rank() OVER (PARTITION BY university.univ_id ORDER BY student.stipend DESC) AS anon_1, university.univ_name, student.kurs, student.surname, student.name, student.stipend 
FROM university JOIN student ON student.univ_id = university.univ_id 
WHERE university.univ_name IN (__[POSTCOMPILE_univ_name_1]) GROUP BY university.univ_id, student.student_id
20


[(1, 'ВГУ', 2, 'Зайцева', 'Ольга', 250),
 (1, 'ВГУ', 5, 'Белкин', 'Вадим', 250),
 (1, 'ВГУ', 2, 'Некрасова', 'Вероника', 250),
 (2, 'ВГУ', 3, 'Петров', 'Петр', 200),
 (2, 'ВГУ', 3, 'Жданова', 'Ирина', 200),
 (2, 'ВГУ', 3, 'Лукин', 'Артем', 200),
 (3, 'ВГУ', 1, 'Иванов', 'Иван', 150),
 (3, 'ВГУ', 1, 'Агафонов', 'Василий', 150),
 (3, 'ВГУ', 1, 'Борисова', 'Марина', 150),
 (3, 'ВГУ', 1, 'Аксенов', 'Валерий', 150),
 (4, 'ВГУ', 5, 'Сергеева', 'Фёкла', 100),
 (4, 'ВГУ', 5, 'Мельникова', 'Дарья', 100),
 (4, 'ВГУ', 4, 'Федоров', 'Гостомысл', 100),
 (5, 'ВГУ', 2, 'Кузнецов', 'Борис', 0),
 (5, 'ВГУ', 3, 'Семенова', 'Вероника', 0),
 (5, 'ВГУ', 3, 'Павлов', 'Андрей', 0),
 (1, 'НГУ', 5, 'Трофимов', 'Владислав', 250),
 (2, 'НГУ', 1, 'Веселов', 'Тимур', 150),
 (3, 'НГУ', 4, 'Журавлева', 'Вера', 0),
 (3, 'НГУ', 3, 'Жуков', 'Пров', 0)]

## 4. Всех студентов, сдававших экзамены, ранжируйте (без пропуска значений ранга) по убыванию средней оценки. Результат должен содержать номер ранга, фамилию, имя студента, среднюю оценку.

In [57]:
stmt = select(func.dense_rank().over( order_by = desc(func.avg(exam_marks.c.mark))), student.c.surname, student.c.name, func.avg(exam_marks.c.mark))
stmt = stmt.select_from(student).join(exam_marks, exam_marks.c.student_id == student.c.student_id)
stmt = stmt.group_by(student.c.student_id)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result

SELECT dense_rank() OVER (ORDER BY avg(exam_marks.mark) DESC) AS anon_1, student.surname, student.name, avg(exam_marks.mark) AS avg_1 
FROM student JOIN exam_marks ON exam_marks.student_id = student.student_id GROUP BY student.student_id
37


[(1, 'Борисова', 'Марина', 5.0),
 (1, 'Блохина', 'Ираида', 5.0),
 (1, 'Жданова', 'Ирина', 5.0),
 (1, 'Лапина', 'Элеонора', 5.0),
 (1, 'Белкин', 'Вадим', 5.0),
 (1, 'Кузнецов', 'Борис', 5.0),
 (2, 'Лукин', 'Артем', 4.0),
 (2, 'Васильев', 'Матвей', 4.0),
 (2, 'Лазарев', 'Константин', 4.0),
 (2, 'Мельникова', 'Светлана', 4.0),
 (2, 'Костина', 'Кира', 4.0),
 (2, 'Жук', 'Оксана', 4.0),
 (2, 'Пак', 'Ирина', 4.0),
 (2, 'Семенова', 'Вероника', 4.0),
 (2, 'Котов', 'Павел', 4.0),
 (2, 'Сидоров', 'Вадим', 4.0),
 (3, 'Зайцева', 'Ольга', 3.5),
 (4, 'Петрова', 'Екатерина', 3.0),
 (4, 'Антонов', 'Анатолий', 3.0),
 (4, 'Кондрат', 'Синицин', 3.0),
 (5, 'Казакова', 'Мария', 2.5),
 (6, 'Вячеслав', 'Пахомов', 2.0),
 (6, 'Веселов', 'Тимур', 2.0),
 (6, 'Трофимов', 'Владислав', 2.0),
 (6, 'Калашников', 'Евгений', 2.0),
 (6, 'Мельникова', 'Дарья', 2.0),
 (6, 'Ордуханян', 'Рафаэль', 2.0),
 (7, 'Жуков', 'Пров', 1.0),
 (7, 'Федоров', 'Гостомысл', 1.0),
 (7, 'Суханов', 'Константин', 1.0),
 (7, 'Вишнякова', 'Анжел

## 5. Для каждого университета узнайте, сколько % студентов на каждом курсе университета от общего числа студентов университета. В результате покажите название университета, номер курса, количество студентов на курсе, долю студентов университета на курсе в %.

In [68]:
s1 = student.alias("s1")

subq = select(func.count(s1.c.student_id))
subq = subq.where(s1.c.univ_id == university.c.univ_id).scalar_subquery()

stmt = select(func.dense_rank().over(partition_by = university.c.univ_id, order_by = func.count(student.c.student_id)), university.c.univ_name, student.c.kurs,  func.count(student.c.student_id), subq, func.round(func.count(student.c.student_id)*100.0/subq, 2))
stmt = stmt.select_from(university).join(student, student.c.univ_id == university.c.univ_id)
stmt = stmt.group_by(university.c.univ_id,student.c.kurs)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:15]

SELECT dense_rank() OVER (PARTITION BY university.univ_id ORDER BY count(student.student_id)) AS anon_1, university.univ_name, student.kurs, count(student.student_id) AS count_1, (SELECT count(s1.student_id) AS count_2 
FROM student AS s1 
WHERE s1.univ_id = university.univ_id) AS anon_2, round((count(student.student_id) * :count_3) / (SELECT count(s1.student_id) AS count_2 
FROM student AS s1 
WHERE s1.univ_id = university.univ_id), :round_2) AS round_1 
FROM university JOIN student ON student.univ_id = university.univ_id GROUP BY university.univ_id, student.kurs
44


[(1, 'ВГУ', 4, 1, 16, 6.25),
 (2, 'ВГУ', 2, 3, 16, 18.75),
 (2, 'ВГУ', 5, 3, 16, 18.75),
 (3, 'ВГУ', 1, 4, 16, 25.0),
 (4, 'ВГУ', 3, 5, 16, 31.25),
 (1, 'НГУ', 1, 1, 4, 25.0),
 (1, 'НГУ', 3, 1, 4, 25.0),
 (1, 'НГУ', 4, 1, 4, 25.0),
 (1, 'НГУ', 5, 1, 4, 25.0),
 (1, 'БГУ', 1, 1, 9, 11.11),
 (1, 'БГУ', 2, 1, 9, 11.11),
 (2, 'БГУ', 4, 2, 9, 22.22),
 (2, 'БГУ', 5, 2, 9, 22.22),
 (3, 'БГУ', 3, 3, 9, 33.33),
 (1, 'ТГУ', 1, 1, 2, 50.0)]

## 6. Для каждого университета покажите долю в % числа студентов каждого курса от максимального числа студентов на курсе университета.

## 7. Для каждого курса соответствующего университета рассчитайте относительный прирост средней экзаменационной оценки, упорядочив по номеру курса. Результат должен содержать: название университета, номер курса, среднюю оценку на курсе данного университета, значение относительного цепного прироста средней оценки.

In [91]:

stmt = select(func.lag(func.avg(exam_marks.c.mark)).over(partition_by = university.c.univ_id, order_by = student.c.kurs).label("prev_avg_mark"), university.c.univ_name, student.c.kurs, func.avg(exam_marks.c.mark).label("avg_mark"))
stmt = stmt.select_from(university).join(student, student.c.univ_id == university.c.univ_id)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id)
stmt = stmt.group_by(university.c.univ_id,student.c.kurs).subquery()

stmt = select(stmt.c.univ_name, stmt.c.kurs, func.round(stmt.c.avg_mark, 2),func.round(stmt.c.prev_avg_mark, 2), func.round(stmt.c.avg_mark*100.0/stmt.c.prev_avg_mark - 100,2))

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:15]

SELECT anon_1.univ_name, anon_1.kurs, round(anon_1.avg_mark, :round_2) AS round_1, round(anon_1.prev_avg_mark, :round_4) AS round_3, round((anon_1.avg_mark * :avg_mark_1) / anon_1.prev_avg_mark - :param_1, :round_6) AS round_5 
FROM (SELECT lag(avg(exam_marks.mark)) OVER (PARTITION BY university.univ_id ORDER BY student.kurs) AS prev_avg_mark, university.univ_name AS univ_name, student.kurs AS kurs, avg(exam_marks.mark) AS avg_mark 
FROM university JOIN student ON student.univ_id = university.univ_id JOIN exam_marks ON exam_marks.student_id = student.student_id GROUP BY university.univ_id, student.kurs) AS anon_1
29


[('ВГУ', 1, 3.0, None, None),
 ('ВГУ', 2, 4.0, 3.0, 33.33),
 ('ВГУ', 3, 4.33, 4.0, 8.33),
 ('ВГУ', 4, 1.0, 4.33, -76.92),
 ('ВГУ', 5, 2.5, 1.0, 150.0),
 ('НГУ', 1, 2.0, None, None),
 ('НГУ', 3, 1.0, 2.0, -50.0),
 ('НГУ', 5, 2.0, 1.0, 100.0),
 ('БГУ', 3, 1.5, None, None),
 ('БГУ', 4, 3.5, 1.5, 133.33),
 ('БГУ', 5, 4.0, 3.5, 14.29),
 ('ТГУ', 1, 4.0, None, None),
 ('ТГУ', 4, 4.0, 4.0, 0.0),
 ('ВГМА', 1, 2.0, None, None),
 ('ВГМА', 2, 1.0, 2.0, -50.0)]